In [15]:
! pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/cpu/x86_64/mindspore-2.2.14-cp37-cp37m-win_amd64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/cpu/x86_64/mindspore-2.2.14-cp37-cp37m-win_amd64.whl (74.9 MB)


In [1]:
import mindspore
mindspore.set_context(device_target='CPU')
mindspore.run_check()

MindSpore version:  2.2.14
The result of multiplication calculation is correct, MindSpore has been installed on platform [CPU] successfully!


In [8]:
! pip install --upgrade mindspore==2.2.14 --user

In [8]:
! pip install mindvision

     ------------------------------------ 194.1/194.1 kB 510.8 kB/s eta 0:00:00


     --------------------------------------- 45.2/45.2 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 7.1/7.1 MB 7.6 MB/s eta 0:00:00
     -------------------------------------- 325.3/325.3 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 38.5/38.5 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94587 sha256=0155c4e5958dcb69976891b54763d3c31d56591cbf552af69d19f55386c44ee3
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\6a\1f\28\926877e62580e3ef78944c6b21b40ed3d300b0b83a7fa925ac
Successfully built ml-collections
  Attempting uninstall: scikit-learn
    Found existing 

In [ ]:
! pip install visualdl

In [31]:
# 下载mnist数据集
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)



file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:01<00:00, 7.22MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [14]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset


In [34]:
from visualdl import LogWriter

# 初始化日志写入器，指定日志文件存放路径
log_writer = LogWriter(logdir="log", name = "train_log")

# 可以使用 log_writer 来记录数据

In [16]:
# 定义记录损失和准确率的辅助函数
def add_scalar(tag, step, value):
    log_writer.add_scalar(tag=tag, step=step, value=value)

In [17]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')


In [18]:
print(train_dataset.get_col_names())

['image', 'label']


In [19]:
# 数据处理流水线，使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小64的批次，送入模型进行训练。
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)
    
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


In [20]:
# 转换数据集
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

In [23]:
import mindspore.nn as nn
from mindspore.common.initializer import Normal

class LeNet5(nn.Cell):
    def __init__(self, num_class=10):
        super(LeNet5, self).__init__()
        # weight_init = Normal(0.02)
        self.conv1 = nn.Conv2d(1, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        
        # 根据卷积层和池化层后的数据尺寸计算全连接层的输入特征数
        # 假设输入图像尺寸为 28x28
        # conv1 后尺寸变为 24x24，池化后变为 12x12
        # conv2 后尺寸变为 4x4，池化后仍然为 4x4（因为已经是最小尺寸）
        self.in_channels_fc1 = 16 * 4 * 4  # conv2 后尺寸变为 4x4，通道数为 16
        
        self.fc1 = nn.Dense(self.in_channels_fc1, 120)
        self.fc2 = nn.Dense(120, 84)
        self.fc3 = nn.Dense(84, num_class)
        
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)  # conv1 后尺寸变为 24x24，池化后变为 12x12
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)  # conv2 后尺寸变为 4x4（因为有 'valid' 填充）
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [24]:
model = LeNet5()

# from mindvision.classification.models import lenet
# model = lenet(num_classes=10, pretrained=False)
print(model)

LeNet5<
  (conv1): Conv2d<input_channels=1, output_channels=6, kernel_size=(5, 5), stride=(1, 1), pad_mode=valid, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=<mindspore.common.initializer.HeUniform object at 0x00000203F458FB08>, bias_init=None, format=NCHW>
  (conv2): Conv2d<input_channels=6, output_channels=16, kernel_size=(5, 5), stride=(1, 1), pad_mode=valid, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=<mindspore.common.initializer.HeUniform object at 0x00000203FB54B688>, bias_init=None, format=NCHW>
  (fc1): Dense<input_channels=256, output_channels=120, has_bias=True>
  (fc2): Dense<input_channels=120, output_channels=84, has_bias=True>
  (fc3): Dense<input_channels=84, output_channels=10, has_bias=True>
  (relu): ReLU<>
  (max_pool2d): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
  (flatten): Flatten<>
  >


In [25]:
# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

# 定义前向函数
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

# 定义梯度函数
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# 定义一步训练以及训练函数
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            add_scalar("loss_100batches", batch, loss)
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")



In [26]:
# 定义测试函数
def test(model, dataset, loss_fn, epoch):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    # 记录测试损失和准确率
    add_scalar("test_loss", epoch, test_loss)
    add_scalar("accuracy", epoch, correct * 100)  # 准确率记录为百分比
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [37]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(model, train_dataset)
    test(model, test_dataset, loss_fn, epoch)
print("Done!")
log_writer.close()


Epoch 1
-------------------------------
loss: 0.057766  [  0/938]
loss: 0.018664  [100/938]
loss: 0.015333  [200/938]
loss: 0.005780  [300/938]
loss: 0.030585  [400/938]
loss: 0.058302  [500/938]
loss: 0.011245  [600/938]
loss: 0.023308  [700/938]
loss: 0.022798  [800/938]
loss: 0.022768  [900/938]
Test: 
 Accuracy: 98.6%, Avg loss: 0.045053 

Epoch 2
-------------------------------
loss: 0.025303  [  0/938]
loss: 0.020217  [100/938]
loss: 0.040578  [200/938]
loss: 0.007742  [300/938]
loss: 0.034534  [400/938]
loss: 0.013235  [500/938]
loss: 0.003862  [600/938]
loss: 0.044205  [700/938]
loss: 0.024428  [800/938]
loss: 0.028854  [900/938]
Test: 
 Accuracy: 98.9%, Avg loss: 0.034058 

Epoch 3
-------------------------------
loss: 0.007700  [  0/938]
loss: 0.020927  [100/938]
loss: 0.006692  [200/938]
loss: 0.042321  [300/938]
loss: 0.001679  [400/938]
loss: 0.063423  [500/938]
loss: 0.020724  [600/938]
loss: 0.015198  [700/938]
loss: 0.051185  [800/938]
loss: 0.013300  [900/938]
Test: 
 

In [38]:
mindspore.save_checkpoint(model, "./lenet/lenet5.ckpt")